# GPT-5を使ったフロントエンド開発

GPT-5はフロントエンド開発において大きな飛躍をもたらします。このモデルは、フルスタックアプリケーションを一度で開発し、複雑なリファクタリングを簡単に見せ、大規模なコードベース内で外科的な編集を行うことに優れていることが確認されています。

このクックブックでは、GPT-5を使用してフロントエンドアプリケーションを開発する際の例と学習内容を、複数の観点から紹介します。

## はじめに
強力なフロントエンドアプリケーションを開発する上で効果的であることが確認されている一般的な原則があります。これらの学習内容の一部を[プロンプトガイド](https://cookbook.openai.com/examples/gpt-5/gpt-5_prompting_guide)で共有しています。以下は、フロントエンドアプリケーションを構築する際に考慮すべき重要な要素です。

モデルを適切に導くために推奨するライブラリとパッケージは以下の通りです：
- フレームワーク: Next.js (TypeScript)、React、HTML
- スタイリング / UI: Tailwind CSS、shadcn/ui、Radix Themes
- アイコン: Material Symbols、Heroicons、Lucide
- アニメーション: Motion
- フォント: San Serif、Inter、Geist、Mona Sans、IBM Plex Sans、Manrope

これらのパッケージは網羅的なリストではなく、多様なアプリケーションスタイルを確認しています。

以下では、API上でフロントエンドの抽象化を簡単に反復する方法を紹介します。ユーザーがGPT-5でどのように創造性を解き放つことができるかを見るのを楽しみにしています。

## インタラクティブな例

ゼロからフロントエンドを作成する例を詳しく見てみましょう。まず、GPT-5から生成されたウェブサイトを確認するためのヘルパー関数をいくつか作成しましょう。

In [ ]:
import os
import re
import webbrowser
from pathlib import Path

import openai
from openai.types.responses import ResponseInputParam

client = openai.OpenAI()


def get_response_output_text(input: str | ResponseInputParam):
    
    response = client.responses.create(
        model="gpt-5",
        input=input,
    )
    return response.output_text


def extract_html_from_text(text: str):
    """Extract an HTML code block from text; fallback to first code block, else full text."""
    html_block = re.search(r"```html\s*(.*?)\s*```", text, re.DOTALL | re.IGNORECASE)
    if html_block:
        result = html_block.group(1)
        return result
    any_block = re.search(r"```\s*(.*?)\s*```", text, re.DOTALL)
    if any_block:
        result = any_block.group(1)
        return result
    return text


def save_html(html: str, filename: str) -> Path:
    """Save HTML to outputs/ directory and return the path."""
    try:
        base_dir = Path(__file__).parent
    except NameError:
        base_dir = Path.cwd()

    folder = "outputs"
    outputs_dir = base_dir / folder
    outputs_dir.mkdir(parents=True, exist_ok=True)

    output_path = outputs_dir / filename
    output_path.write_text(html, encoding="utf-8")
    return output_path

def open_in_browser(path: Path) -> None:
    """Open a file in the default browser (macOS compatible)."""
    try:
        webbrowser.open(path.as_uri())
    except Exception:
        os.system(f'open "{path}"')



それでは、上記を1つのヘルパー関数にまとめてみましょう。

In [7]:
def make_website_and_open_in_browser(*, website_input: str | ResponseInputParam, filename: str = "website.html"):
    response_text = get_response_output_text(website_input)
    html = extract_html_from_text(response_text)
    output_path = save_html(html, filename)
    open_in_browser(output_path)

簡単な例から始めましょう：適切なテーマでレトロゲームストアをワンショットで構築する

In [8]:
make_website_and_open_in_browser(
    website_input="Make me landing page for a retro-games store. Retro-arcade noir some might say",
    filename="retro_dark.html",
)

get_response: finished
extract_html_from_text: finished (raw text)
save_html: finished -> outputs/retro_dark.html
open_in_browser: finished


一行のワンショットプロンプトにしては悪くありませんね！

<img src="../../images/retro_dark.png" style="width:60vw; display:block; margin:auto;">

今度はもう少し明るく、そして少し柔らかい感じに調整してみましょう。

In [9]:
make_website_and_open_in_browser(
    website_input="Make me landing page for a retro-games store. Make it light, more pastel coloured & flowery (think Mario, not cyberpunk)", 
    filename="retro_light.html"
)

get_response: finished
extract_html_from_text: finished (raw text)
save_html: finished -> outputs/retro_light.html
open_in_browser: finished


ご覧の通り、GPT-5は非常に操縦しやすく、たった一行でアプリケーション全体を簡単に変更することができます。

<img src="../../images/retro_light.png" style="width:60vw; display:block; margin:auto;">

しかし、既存のウェブサイトデザインに追加を行いたい場合はどうでしょうか？例えば、私たちは既にこのダッシュボードを持っています。

<img src="../../images/input_image.png" style="width:60vw; display:block; margin:auto;">

GPT-5はネイティブにマルチモーダルであり、画像とテキストの両方の入力を受け付けるため、フロントエンドアプリケーションを生成する際に、画像入力を活用してモデルのパフォーマンスを向上させることができます。

In [10]:
import base64
from openai.types.responses import ResponseInputImageParam

# Function to encode the image
def encode_image(image_path: str):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

image_path="../../images/input_image.png"
encoded_image = encode_image(image_path)
input_image: ResponseInputImageParam = {"type": "input_image", "image_url": f"data:image/png;base64,{encoded_image}", "detail": "auto"}
input: ResponseInputParam = [
    {
        "role": "user",
        "content": [
            {"type": "input_text", "text": "Can you make a login page for this website that maintains the same theme"},
            input_image,
        ],
    }
]

make_website_and_open_in_browser(
    website_input=input, 
    filename="login_page.html"
)

get_response: finished
extract_html_from_text: finished (raw text)
save_html: finished -> outputs/login_page.html
open_in_browser: finished


ご覧の通り、GPT-5はアプリの既存のスタイルと雰囲気に見事にマッチしています。

<img src="../../images/login_page.png" style="width:60vw; display:block; margin:auto;">

これまでは非常に静的でした - もっとインタラクティブなタスクを試してみましょう

In [11]:
make_website_and_open_in_browser(
    website_input="Make me a snake game. It should be futuristic, neon, cyberpunk style. Make sure the typography is suitably cool.", 
    filename="snake_game.html"
)

get_response: finished
extract_html_from_text: finished (raw text)
save_html: finished -> outputs/snake_game.html
open_in_browser: finished


テーマに一貫したスネークゲームができました：色、タイポグラフィ、さらには音まで統一されています

<img src="../../images/snake_game.png" style="width:60vw; display:block; margin:auto;">

GPT-5がフロントエンド開発においてどれほど強力であるかについて、いくつかのアイデアをお伝えできたことを願っています。単一の曖昧な指示とAPIコールから、本番環境レベルの出力を得ることができます。

今度はあなたの番です - あなたが何を構築するのか、私たちは楽しみに待っています。

In [12]:
your_prompt = "[edit this! what website would you like to build?]"

make_website_and_open_in_browser(
    website_input=your_prompt, 
    filename="your_website.html"
)

get_response: finished
extract_html_from_text: finished (raw text)
save_html: finished -> outputs/your_website.html
open_in_browser: finished
